# Inspecting some test bounding boxes

This notebook is meant to be a visual sanity check on datasets and signal generation by making some signals with known contents and viewing their spectrograms.

---

In [ ]:
# Variables
from torchsig.transforms.transforms import Spectrogram
from torchsig.transforms.metadata_transforms import YOLOLabel
from torchsig.signals.signal_lists import TorchSigSignalLists
from tqdm.notebook import tqdm

root = "./datasets/detector_example"
fft_size = 1024
num_iq_samples_dataset = fft_size**2
class_list = TorchSigSignalLists.all_signals
num_classes = len(class_list)
num_train = 20  # size of train dataset
num_val = 5  # size of validation dataset

# transform data into a spectrogram image
transforms = [Spectrogram(fft_size=fft_size), YOLOLabel()]
# YOLO labels are expected to be (class index, x center, y center, width, height)
# all normalized to zero, with (0,0) being upper left corner

In [ ]:
from torchsig.datasets.datasets import TorchSigIterableDataset
from torchsig.utils.defaults import default_dataset

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

dataset = default_dataset(
    noise_power_db=-4.0,
    num_signals_min=6,
    num_signals_max=6,
    num_iq_samples_dataset=num_iq_samples_dataset,
    fft_size=fft_size,
    impairment_level=None,
    transforms=transforms,
    target_labels=["yolo_label"],
)
dataset.seed(10)
class_list = [
    gen.family_name if "family_name" in gen.keys() else gen.class_name
    for gen in dataset.signal_generators
]

In [ ]:
# show sample from dataset
data, label = next(dataset)
print(f"Data shape: {data.shape}")
print(f"Number of signals: {len(label)}")
nl = "\n"
print(f"Labels:\n{nl.join(str(l) for l in label)}")

height, width = data.shape
fig = plt.figure(figsize=(12, 6))
fig.tight_layout()

ax = fig.add_subplot(1, 1, 1)
pos = ax.imshow(data, aspect="auto", cmap="Wistia", vmin=dataset.noise_power_db)

fig.colorbar(pos, ax=ax)


for t in label:
    classindex, xcenter, ycenter, normwidth, normheight = t
    actualwidth = width * normwidth
    actualheight = height * normheight
    actualxcenter = xcenter * width
    actualycenter = ycenter * height
    x_lowerleft = actualxcenter - (actualwidth / 2)
    y_lowerleft = actualycenter + (actualheight / 2)
    ax.add_patch(
        Rectangle(
            (x_lowerleft, y_lowerleft),
            actualwidth,
            -actualheight,
            linewidth=1,
            edgecolor="blue",
            facecolor="none",
        )
    )
    textDisplay = str(class_list[classindex])
    ax.text(
        x_lowerleft,
        y_lowerleft,
        textDisplay,
        bbox=dict(facecolor="w", alpha=0.5, linewidth=0),
    )

In [ ]:
# show sample from dataset
data, label = next(dataset)
print(f"Data shape: {data.shape}")
print(f"Number of signals: {len(label)}")
nl = "\n"
print(f"Labels:\n{nl.join(str(l) for l in label)}")

height, width = data.shape
fig = plt.figure(figsize=(12, 6))
fig.tight_layout()

ax = fig.add_subplot(1, 1, 1)
pos = ax.imshow(data, aspect="auto", cmap="Wistia", vmin=dataset.noise_power_db)

fig.colorbar(pos, ax=ax)


for t in label:
    classindex, xcenter, ycenter, normwidth, normheight = t
    actualwidth = width * normwidth
    actualheight = height * normheight
    actualxcenter = xcenter * width
    actualycenter = ycenter * height
    x_lowerleft = actualxcenter - (actualwidth / 2)
    y_lowerleft = actualycenter + (actualheight / 2)
    ax.add_patch(
        Rectangle(
            (x_lowerleft, y_lowerleft),
            actualwidth,
            -actualheight,
            linewidth=1,
            edgecolor="blue",
            facecolor="none",
        )
    )
    textDisplay = str(class_list[classindex])
    ax.text(
        x_lowerleft,
        y_lowerleft,
        textDisplay,
        bbox=dict(facecolor="w", alpha=0.5, linewidth=0),
    )